In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from joblib import load

In [3]:
typhi_grna = "./gRNA_sequences/typhi_grna.csv"
typhi_targets = "./gRNA_sequences/typhi_targets.csv"
typhi_records = pd.read_csv(typhi_grna)
typhi_records_targets = pd.read_csv(typhi_targets)
typhi_records['gRNA'] = typhi_records['gRNA'].str.lower()
typhi_records_targets['Sequence'] = typhi_records_targets['Sequence'].str.lower()
typhi_records.gRNA[0],typhi_records.Type[0],typhi_records_targets.Sequence[0],typhi_records_targets.Target[0]


('aaagataccagagcccgaat',
 'sty1607',
 'tttaaaagcatttgcggcagagaaaggacttgacggcgctgaaaaagatgcatttcttaagagtgcgtttgaacacttacatgcactaagtaaggctggtgaaccacttagtcttgaaacccttgttaatgccatttggccccaggctcccgaggagttatctggtaaacttgctgctgaagagttggagttatcggatggcttcgttcctgatggtcgagtcattcgggctctggtatcttttaaaggcaagtctaaatattgggaacttaagtttgaccgggaagggaaaactgaaggctatattgattatgatccggaaacaaatatt',
 'sty1607')

In [4]:
#Find best alignment for gRNA against target 
def align(target, grna):
    max = 0
    #For every starting position of target
    for k in range(len(target)):
      count = 0
      #For every nucleotide of the gRNA sequence 
      for l in range(min(len(target[k:]),len(grna))):
        if target[k+l] == grna[l]:
          count += 1
      if count >= max:
        max = count
        position = k
    if position < 20 or position + 40 > len(target):
      return None,0
    return target[position-20:position+40],max

In [5]:
targets = []
hamming = []
for k in range(len(typhi_records)):
    target,max = align(typhi_records_targets[typhi_records_targets['Target']==typhi_records.Type[k]]['Sequence'].tolist()[0],typhi_records.gRNA[k])
    
    targets.append(target)
    hamming.append(20-max)

In [6]:
#Import features from Random Forest Model
clf = load('Models/grnamodel_ds.joblib')
output_ds = pd.read_pickle("./Outputs/df_feat_ds_doubletree.pkl") 
output_ds = output_ds[output_ds.importance_values>0.01]
output_ds

,features,importance_values
690,Motif: ta,0.079278
625,gc_target,0.040530
258,"targetposition41, g",0.024618
647,Motif: c,0.022447
685,Motif: gta,0.019166
634,Motif: acc,0.018614
627,Motif: aa,0.018292
668,Motif: g,0.023565
653,Motif: cc,0.016772
629,Motif: aac,0.015493


In [7]:
def one_hot_encode(seq):
    mapping = dict(zip("acgt", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]
def one_hot_encode_duplets(seq):
    mapping = dict(zip(('aa', 'ac', 'ag', 'at', 'ca', 'cc', 'cg', 'ct', 'ga', 'gc', 'gg', 'gt', 'ta', 'tc', 'tg', 'tt'), range(16)))
    seq2 = []
    for i in range(len(seq)-1):
      seq2.append(mapping[seq[i:i+2]])
    return np.eye(16)[seq2]
def calcGC(seq):
  gc_content = [] 
  count = 0
  for l in range(len(seq)):
    if seq[l] == 'g' or seq[l] == 'c':
      count += 1
  return count/len(seq)
def countMotifs(seq, motifs):
  counts = []
  for motif in motifs:
    count = 0
    for i in range(len(seq) - len(motif)):
      if seq[i:i+len(motif)] == motif:
        count += 1
    counts.append(count/(len(seq) - len(motif)))
  return counts
bases = ['a','c','g','t']
motifs_to_look_for = []
for i in range(len(bases)):
  motifs_to_look_for.append(bases[i])
  for j in range(len(bases)):
    motifs_to_look_for.append(bases[i]+bases[j])
    for k in range(len(bases)):
      motifs_to_look_for.append(bases[i]+bases[j]+bases[k])
      # for l in range(len(bases)):
      #   motifs_to_look_for.append(bases[i]+bases[j]+bases[k]+bases[l])
motifs_to_look_for.append('aaaa')
motifs_to_look_for.append('cccc')
motifs_to_look_for.append('gggg')
motifs_to_look_for.append('tttt')

In [10]:
X_ds = []
true_typhi_records = []
true_typhi_type = []
true_typhi_hamming = []
true_typhi_target = []
k = 20
for i in range(len(typhi_records)):
  ## 1 is not length 20 so ignore
  if len(typhi_records.gRNA[i])!=20:
    continue
  # if no suitable target, ignore
  if targets[i] == None:
    continue
  else: 
    true_typhi_records.append(typhi_records.gRNA[i])
    true_typhi_type.append(typhi_records.Type[i])
    true_typhi_hamming.append(hamming[i])
    true_typhi_target.append(targets[i])
  t = np.concatenate((one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(targets[i][:k]), one_hot_encode(targets[i][k+20:]), one_hot_encode_duplets(typhi_records.gRNA[i])
                           , calcGC(typhi_records.gRNA[i]), calcGC(targets[i]), countMotifs(typhi_records.gRNA[i], motifs_to_look_for)), axis = None)
  X_ds.append(t)
X_reduced_ds = np.array(X_ds)[:,output_ds.index]

In [11]:
y_pred = clf.predict(X_reduced_ds)
y_pred

array([0.79048256, 1.27299129, 1.46244908, 1.20427602, 0.88743979,
       1.3117191 , 1.49911074, 1.11136456, 1.00763867, 1.45203424,
       1.50482382, 1.4569847 , 1.55312039, 1.50326174, 1.63830278,
       1.27255936, 1.47997963, 1.45820621, 1.18924223, 0.98300813,
       1.91012694, 1.36243569, 1.28754787, 1.52701427, 1.7319218 ,
       1.65589646, 1.59790861, 1.74708678, 1.1359112 , 1.49231727,
       1.57410418, 1.11551522, 0.73397334, 1.2598804 , 0.83993857,
       1.6615826 , 1.07664922, 1.03859549, 1.73703967, 0.90525568,
       1.24784044, 1.2732719 , 1.25328468, 1.46572586])

In [13]:
for i in range(len(y_pred)):
    if true_typhi_hamming[i]!=0:
        y_pred[i] = 0
 

In [14]:
df = pd.DataFrame({'gRNA':true_typhi_records, 'my version of target':true_typhi_target, 'type':true_typhi_type, 'hamming distance':true_typhi_hamming,'predicted_ds_k':y_pred})
df = df.sort_values('predicted_ds_k', ascending = False)

df.to_csv('./Outputs/typhi_kvalues_prediction.csv')
df

,gRNA,my version of target,type,hamming distance,predicted_ds_k
20,gtagcttactacgccgattg,cttagataaaagttattttggtagcttactacgccgattgaacaaa...,sty2879,0,1.910127
38,agacagaactgggtaataaa,gcgtgatccgactaaatttcagacagaactgggtaataaaaaaggt...,sty0307,0,1.737040
24,aaagtcctgattttgttgat,tcataattcatcatcttttgaaagtcctgattttgttgattcgaaa...,sty0307,0,1.731922
25,ttgattcgaaagatttctat,ttttgaaagtcctgattttgttgattcgaaagatttctatgcaggc...,sty0307,0,1.655896
26,tatgcaggcattggttatga,ttgttgattcgaaagatttctatgcaggcattggttatgaccaggg...,sty0307,0,1.597909
30,aaaatacctgtgcggcccgt,gaaacttggtgctcagtttgaaaatacctgtgcggcccgtatgagc...,sty0307,0,1.574104
23,ccgttgttgatattaaacaa,cttaaaaagtgatttttttgccgttgttgatattaaacaatttttt...,sty2879,0,1.527014
10,agtcaacgagccgtagtcgt,tgatattaaacaattttttcagtcaacgagccgtagtcgtatcacc...,sty2879,0,1.504824
16,aactgtaagaaacttgtccc,cgggaaatctcaacattttcaactgtaagaaacttgtcccatagtc...,sty2879,0,1.479980
2,ttaagagtgcgtttgaacac,cgctgaaaaagatgcatttcttaagagtgcgtttgaacacttacat...,sty1607,0,1.462449
